In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import datasets
from sklearn import preprocessing
import numpy as np
import pandas as pd
from datetime import datetime
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score

Populating the interactive namespace from numpy and matplotlib


/home/rodrigo/ENV/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
data_set_complete = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/data_set_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13208286 entries, 0 to 13208285
Data columns (total 16 columns):
sepostulo               int8
idaviso                 int64
idpostulante            object
fechapostulacion        datetime64[ns]
ultima_vista            datetime64[ns]
veces_visto             int16
estudios                category
estado                  category
sexo                    category
edad                    int8
titulo                  object
nombre_zona             category
tipo_de_trabajo         category
nivel_laboral           category
nombre_area             category
denominacion_empresa    object
dtypes: category(7), datetime64[ns](2), int16(1), int64(1), int8(2), object(3)
memory usage: 755.8+ MB


In [3]:
def df_encode_categorical_vars(df):
    for column in df.columns:
        if (df[column].dtype.name == 'category'):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column].astype(str))

In [4]:
def df_encode_object_vars(df):
    for column in df.columns:
        if (df[column].dtype == type(object)):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column].astype(str))

In [5]:
muestra = data_set_complete.sample(1000000)

In [6]:
df_encode_categorical_vars(muestra)

In [7]:
muestra.drop(columns=['idaviso','idpostulante','fechapostulacion','ultima_vista','titulo','denominacion_empresa'], inplace=True)

In [8]:
muestra['sepostulo'].value_counts()

1    500441
0    499559
Name: sepostulo, dtype: int64

In [9]:
train_set, test_set, train_classes, test_classes = train_test_split(muestra, muestra['sepostulo'], test_size=0.3)

In [10]:
train_set.drop(columns=['sepostulo'], inplace=True)
test_set.drop(columns=['sepostulo'], inplace=True)
train_classes.drop(columns=['sepostulo'], inplace=True)
test_classes.drop(columns=['sepostulo'], inplace=True)

/home/rodrigo/ENV/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/rodrigo/ENV/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
knn = KNeighborsClassifier(n_neighbors=7)

In [13]:
knn.fit(train_set, train_classes)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [14]:
knn.score(test_set, test_classes)

0.9608266666666667

In [15]:
test_100k = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/test_final_100k_complete.csv')
test_100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
fechapostulacion        0 non-null float64
ultima_vista            34161 non-null object
veces_visto             100000 non-null int64
estudios                94388 non-null object
estado                  94388 non-null object
sexo                    100000 non-null object
edad                    100000 non-null int64
titulo                  100000 non-null object
nombre_zona             100000 non-null object
tipo_de_trabajo         100000 non-null object
nivel_laboral           95569 non-null object
nombre_area             100000 non-null object
denominacion_empresa    99988 non-null object
dtypes: float64(1), int64(4), object(11)
memory usage: 12.2+ MB


/home/rodrigo/ENV/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
test_100k.drop(columns=['idaviso','idpostulante','fechapostulacion','ultima_vista','titulo','denominacion_empresa'], inplace=True)

In [17]:
df_encode_object_vars(test_100k)

In [18]:
test_100k['sepostulo'] = knn.predict(test_100k.drop(columns='id'))

In [19]:
test_100k['sepostulo'].value_counts()

0    66383
1    33617
Name: sepostulo, dtype: int64

In [20]:
test_100k[['id','sepostulo']].to_csv('Knn_7_con_veces_visto.csv', index=False)